## LP Framework for Analyzing Truncated Distribution Instances: An Implementation

$n=50$ with $50$ distributions in the family now takes around $30$ seconds to run.

Please refer to the proofs at https://www.mathcha.io/editor/xlKD2SYWivrhvz5Yvzc0d4Dp4T241vm8TMxW8kP

### Tasks to do
1. Proof check
2. Code check: A suggestion is to recover the Pareto optimal stopping rule from the LP variables (see Lemma 7) and verify the competitive ratios obtained.
3. Instance design and testing

In [2]:
import numpy as np
from itertools import product
from scipy.optimize import fsolve
from gurobipy import *
from json import loads
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [3]:
# From Hill and Kertz (1982), ~ 0.745 hard instance when n -> infty
def instance(n, eps, print_head_tail=0):
    
    def phi(w,x,n):  # Definition 3.1
        c = n/(n-1)
        return c*w**(1/c)+x/(n-1)

    def eta(a,n):  # Definition 3.1
        arr = [phi(0,a,n)]
        for j in range(1,n+1):
            arr.append(phi(arr[-1],a,n))
        return arr

    def G(a,n):  # Definition 3.3
        return eta(a,n)[-2]

    def ratio(a,n):  # For defining alpha(n) below
        return G(a,n)-1

    def alpha(n):  # Proposition 3.4
        return float(fsolve(ratio,0,n))
    
    # Page 342, between Corollary 4.3 and Proposition 4.4
    a = alpha(n)
    s = [x**(1/n) for x in eta(a,n)[:-2]]+[1.0-eps,1.0]
    p = [s[0]]+[s[i]-s[i-1] for i in range(1,n+1)]
    
    # Lemma 2.5
    v = [0]
    s_prod = [1]
    for i in range(n+1):
        s_prod.append(s_prod[-1]*s[i])
    v.append((1-s[-2])/((1-s[-2])*sum(s_prod[:-3])+s_prod[-3]))
    for i in range(2,n):
        v.append(v[1]*sum(s_prod[:i]))
    v.append(1)
    
    # Definition 3.10 & Statement (8)
    # Take reciprocal for our definition of competitive ratio
    r = 1/(1+a)
    
    return r, v, p

In [4]:
class Distributions:
    """
    A class representing a family of distributions obtained by tail truncation.

    Parameters:
    values (list of float): A nonnegative array of strictly increasing values representing 
                            the support of the non-truncated distribution.
                            
    probs (list of float): A nonnegative array of the same length as `values` whose values sum to 1, 
                           representing the probability masses of the non-truncated distribution.
                           
    indices (list of int): A nonnegative integer array where each element represents the 
                           position of truncation of distribution k.
                           Each value must be between 0 and len(values) - 1.
                           
    prior_index (int): An integer representing the position of truncation of the prior distribution,
                       must be between 0 and len(values) - 1.
    """

    def __init__(self, values, probs, indices, prior_index):
        
        if not all(v >= 0 for v in values):
            raise ValueError("All values must be nonnegative.")
        if not all(values[i] < values[i + 1] for i in range(len(values) - 1)):
            raise ValueError("Values must be strictly increasing.")
        
        if len(probs) != len(values):
            raise ValueError("Probs must have the same length as values.")
        if not all(p >= 0 for p in probs):
            raise ValueError("All probs must be nonnegative.")
        if not (abs(sum(probs) - 1) < 1e-9):  # Allow for floating point precision
            raise ValueError("Probs must sum to 1.")

        if not all(isinstance(index, int) and 0 <= index < len(values) for index in indices):
            raise ValueError("Indices must be nonnegative integers between 0 and len(values) - 1.")
        
        if not (isinstance(prior_index, int) and 0 <= prior_index < len(values)):
            raise ValueError("Prior index must be an integer between 0 and len(values) - 1.")

        self.v = values
        self.p = probs
        self.cum_p = [0.0] + np.cumsum(self.p).tolist()
        self.indices = sorted(set(indices+[prior_index]), reverse=True)
        self.K = len(self.indices)
        self.k_star = self.indices.index(prior_index)
        self.v_partition = []
        p_partition = [0] * self.K
        l = 0
        for i in range(len(self.v)-1, -1, -1):
            if (l+1 <= len(self.indices)-1 and self.indices[l+1] == i):
                l += 1
            self.v_partition.insert(0,l)
            p_partition[l] += self.p[i]
        self.cum_p_partition = np.cumsum(p_partition[::-1])[::-1].tolist() + [0.0]

    def summarize(self, sigfig = 5):
        print("Summary of distributions (*prior):")
        print(f"Support:\n    {[float(f'{v:.{sigfig}g}') for v in self.v]}")
        print("PMF:")
        for k in range(self.K):
            print(f"{'*' if k == self.k_star else ' '}{k}:", end=" ")
            print(
                [float(f'{p/self.cum_p_partition[k]:.{sigfig}g}')*(i <= self.indices[k])
                 for i, p in enumerate(self.p)]
            )
            
    def I(self, x):
        if (x is None):
            return self.K - 1
        max_element = 0
        if (type(x) == int):
            return self.v_partition[x]
        return self.v_partition[max(x)]

    def c(self, k):
        return 1/self.cum_p_partition[k]

    def prob_of_seq_in_state(self, t, l, mode):
        if (mode == ">="):
            if (t == 0):
                return 1
            return self.cum_p_partition[l]**t
        if (mode == "=="):
            if (t == 0):
                return (1 if l == self.K-1 else 0)
            return self.cum_p_partition[l]**t - self.cum_p_partition[l+1]**t

    def q(self, t, l, i):
        if not (self.I(i) >= l):
            raise ValueError("self.I(i) must be at least l")
        if (self.I(i) == l):
            return self.prob_of_seq_in_state(t-1, l, ">=")
        else:
            return self.prob_of_seq_in_state(t-1, l, "==")

In [5]:
# See numerical issues
# https://docs.gurobi.com/projects/optimizer/en/current/concepts/numericguide/tolerances_scaling.html#secnumerictolerancesscaling

class LPModel:
    """
    A class to create and solve a linear programming model using Gurobi.

    Parameters:
    distribution (Distribution): An instance of the Distribution class.
    n (int): Number of random variables in the arrival sequence.
    alpha (float, int or None): A value between 0 and 1, consistency requirement.
    beta (float, int or None): A value between 0 and 1, robustness requirement.
    outputflag (int, 0 or 1): =  0 or 1 for disabling or enabling solver output respectively.

    Note: Exactly one of alpha and beta must be None. The other one must be a float between 0 and 1.
    """

    def __init__(self, n, distribution, alpha, beta, outputflag = 0):
        if not (isinstance(n, int) and n > 0):
            raise ValueError("n must be a positive integer.")
        self.n = n
        self.F = distribution
        self.model = Model()
        self.variables = None
        self.objective = None
        
        self.build(alpha, beta, outputflag)

    def build(self, alpha, beta, outputflag = 0):

        self.model.setParam('OutputFlag', outputflag)
        self.model.setParam('MIPGap', 1e-9)
        
        if not ((alpha is None and isinstance(beta, (float,int)) and 0 <= beta <= 1) or 
                (isinstance(alpha, (float,int)) and 0 <= alpha <= 1 and beta is None)):
            raise ValueError("Exactly one of alpha and beta must be None. The other one must be a float or int between 0 and 1.")
        
        self.S = self.model.addVars(list(product(range(self.n+1), 
                                                 range(self.F.K))
                                        ), lb=0, ub=1, name="S")
        self.L = self.model.addVars(list(product(range(1, self.n+1),
                                                 range(self.F.K))
                                        ), lb=0, name="L")
        if (beta is None):
            self.alpha = float(alpha)
            self.beta = self.model.addVar(name="beta", lb=0, ub=1)
            self.objective = self.model.setObjective(self.beta, GRB.MAXIMIZE)
            print(f"Given alpha = {self.alpha}, maximizing beta ...")
        else:
            self.alpha = self.model.addVar(name="alpha", lb=0, ub=1)
            self.beta = float(beta)
            self.objective = self.model.setObjective(self.alpha, GRB.MAXIMIZE)
            print(f"Given beta = {self.beta}, maximizing alpha ...")

        self.model.addConstr(
            self.alpha * self.OPT(self.F.k_star) <= self.ALG(self.F.k_star, self.L),
            name=f"consistent_{self.F.k_star}"
        )
        
        for k in range(self.F.K):
            self.model.addConstr(
                self.beta * self.OPT(k) <= self.ALG(k, self.L),
                name=f"robust_{k}"
            )

        for t in range(self.n+1):
            for l in range(self.F.K):
                if (t == 0):
                    self.model.addConstr(
                        self.S[t,l] == (1 if l == self.F.K-1 else 0),
                        name=f"order_{(t,l)}"
                    )
                else:
                    self.model.addConstr(
                        self.S[t,l] * self.F.prob_of_seq_in_state(t, l, "==") <= quicksum(
                            self.w(t,l,i) * self.F.p[i] 
                            for i in range(self.F.indices[l]+1)
                        ),
                        name=f"order_{(t,l)}"
                    )

        for t in range(1, self.n+1):
            for l in range(self.F.K):
                for i in range(self.F.indices[l]+1):
                    self.model.addConstr(
                        self.L[t,l] <= quicksum(
                            self.F.p[j]*(
                                max(self.F.v[j],self.F.v[i]) * self.w(t,l,j)
                                -  self.F.v[i] * self.S[t,l] * self.F.q(t,l,j)
                            )
                            for j in range(self.F.indices[l]+1)
                        ),
                        name=f"piece_{(t,l,i)}"
                    )
        
    def OPT(self, k):
        c = self.F.c(k)
        return c ** self.n * sum(
            self.F.v[i] * (self.F.cum_p[i+1] ** self.n - self.F.cum_p[i] ** self.n)
            for i in range(self.F.indices[k]+1)
        )


    def ALG(self, k, L):
        c = self.F.c(k)
        return quicksum(c**t * 
                        quicksum(L[t,l] for l in range(k, self.F.K))
                        for t in range(1, self.n+1)
                       )

    def w(self, t, l, i):
        if not (self.F.I(i) >= l):
            raise ValueError("self.F.I(i) must be at least l")
        if (self.F.I(i) == l):
            return quicksum(
                self.S[t-1, m] * self.F.prob_of_seq_in_state(t-1, m, "==")
                for m in range(l, self.F.K)
            )
        else:
            return self.S[t-1, l] * self.F.prob_of_seq_in_state(t-1, l, "==")

    def write(self, file):
        """Writes the model to a file.
        Args:
            file (str): The full path including the file name to write the model to.
        """
        self.model.model.write(file)
    
    def solve(self):
        model_presolved = self.model.presolve()
        if (model_presolved.NumConstrs == 0):
            print("Numerically unstable!")
        
        self.model.optimize()
        
        status = self.model.status
        if status == GRB.OPTIMAL:
            S, L = {}, {}
            for x in self.model.getVars():
                if (x.VarName[0]=='S'):
                    name = tuple(loads(x.VarName[1:]))
                    S[name] = x.X
                elif (x.VarName[0]=='L'):
                    name = tuple(loads(x.VarName[1:]))
                    L[name] = x.X
                elif (x.VarName == 'alpha'):
                    alpha = x.X
                    beta = self.beta
                elif (x.VarName == 'beta'):
                    beta = x.X
                    alpha = self.alpha
            print(f"Optimal solution found, (alpha, beta) = {(alpha, beta)}")
            return S, L, alpha, beta
        elif status == GRB.INF_OR_UNBD:
            print("Model is infeasible or unbounded.")
        elif status == GRB.TIME_LIMIT:
            print("Time limit reached.")
        elif status == GRB.INTERRUPTED:
            print("Optimization was interrupted.")
        elif status == GRB.UNBOUNDED:
            print("Model is unbounded.")
        elif status == GRB.INFEASIBLE:
            print("Model is infeasible.")
        else:
            print("Optimization was not successful. Status code:", model.status)
        return None, None, None, None

In [6]:
# Distribution family generation using the instance function as an example
n, eps = 10, 1e-4
_, v, p = instance(n, eps)
indices = list(range(len(v)-1, 0, -1))
prior_index = len(v)-1
F = Distributions(v, p, indices, prior_index)
F.summarize()

# Example usage: Given alpha, maximize beta
alpha, beta = 0.7709894975670004, None
model = LPModel(n, F, alpha, beta, outputflag = 0)
S, L, alpha, beta = model.solve()
if S is not None:
    for k in range(model.F.K):
        print(f" {k}: {model.ALG(k, L) / model.OPT(k)}")

# Example usage: Given beta, maximize alpha
alpha, beta = None, 0.45
model = LPModel(n, F, alpha, beta, outputflag = 0)
S, L, alpha, beta = model.solve()
if S is not None:
    for k in range(model.F.K):
        print(f" {k}: {model.ALG(k, L) / model.OPT(k)}")

Summary of distributions (*prior):
Support:
    [0.0, 0.00033238, 0.00056901, 0.00075408, 0.00090769, 0.0010409, 0.0011606, 0.0012713, 0.0013761, 0.0014776, 1.0]
PMF:
*0: [0.71193, 0.07019, 0.047863, 0.037426, 0.030935, 0.026304, 0.02273, 0.019837, 0.017423, 0.015267, 0.0001]
 1: [0.712, 0.070197, 0.047867, 0.037429, 0.030939, 0.026307, 0.022732, 0.019839, 0.017425, 0.015269, 0.0]
 2: [0.72304, 0.071285, 0.04861, 0.03801, 0.031418, 0.026715, 0.023085, 0.020147, 0.017695, 0.0, 0.0]
 3: [0.73606, 0.072569, 0.049485, 0.038694, 0.031984, 0.027196, 0.023501, 0.02051, 0.0, 0.0, 0.0]
 4: [0.75147, 0.074089, 0.050522, 0.039505, 0.032654, 0.027765, 0.023993, 0.0, 0.0, 0.0, 0.0]
 5: [0.76995, 0.07591, 0.051763, 0.040476, 0.033457, 0.028448, 0.0, 0.0, 0.0, 0.0, 0.0]
 6: [0.79249, 0.078133, 0.053279, 0.041661, 0.034436, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
 7: [0.82075, 0.080919, 0.055179, 0.043147, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
 8: [0.85776, 0.084568, 0.057667, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.

In [7]:
def experiment(N):
    eps = 1e-4  # Note: The smaller the value of eps, the harder the instance.
                # However, Gurobi raises an error when this value is too small
                # because of numerical issues
    for n in N:
        print(f"n = {n}:")
        _, v, p = instance(n, eps)
        indices = list(range(len(v)-1, 0, -1))
        prior_index = len(v)-1
        F = Distributions(v, p, indices, prior_index)
        
        # maximum alpha
        alpha, beta = None, 0.0
        model = LPModel(n, F, alpha, beta, outputflag = 0)
        S, L, alpha, beta = model.solve()
        max_alpha = alpha
        
        # minimum alpha (= beta)
        alpha, beta = 0.0, None
        model = LPModel(n, F, alpha, beta, outputflag = 0)
        S, L, alpha, beta = model.solve()
        min_alpha = beta
        
        # maximize beta for each alpha within the range found
        Alpha, Beta = [], []
        m = 10
        for i, alpha in enumerate(np.linspace(min_alpha, max_alpha, m)):
            model = LPModel(n, F, alpha, None, outputflag = 0)
            S, L, alpha, beta = model.solve()
            if S is not None:
                Alpha.append(alpha)
                Beta.append(beta)
        plt.plot(Alpha, Beta, label=f"{n}")
    plt.legend()
    plt.plot([np.exp(-1), 0.745, 0.745], [np.exp(-1), np.exp(-1), 0])
    plt.title(f"Hardness (eps = {eps})")
    plt.show()

In [9]:
# Experiment demo
experiment(list(range(105,125,5)))

n = 105:
Given beta = 0.0, maximizing alpha ...
Optimal solution found, (alpha, beta) = (0.7496326021869071, 0.0)
Given alpha = 0.0, maximizing beta ...
Optimal solution found, (alpha, beta) = (0.0, 0.6517614078109254)
Given alpha = 0.6517614078109254, maximizing beta ...
Optimal solution found, (alpha, beta) = (0.6517614078109254, 0.6517614078109262)
Given alpha = 0.6626359849638123, maximizing beta ...


KeyboardInterrupt: 

In [ ]:
# Example run to show that support extension does help.
# Same distribution family in the earlier example, with an extension of size 5. Compare the results.

n, eps = 10, 1e-4
_, v, p = instance(n, eps)
prior_index = len(v)-1
v.extend([100**i for i in range(1, 6)])
p.extend([eps/(3**i) for i in range(1, 6)])
p = [_/sum(p) for _ in p]
indices = list(range(len(v)-1, 0, -1))
F = Distributions(v, p, indices, prior_index)
F.summarize()

# Example usage: Given alpha, maximize beta
alpha, beta = 0.7685927046871043, None
model = LPModel(n, F, alpha, beta, outputflag = 0)
S, L, alpha, beta = model.solve()
if S is not None:
    for k in range(model.F.K):
        print(f" {k}: {model.ALG(k, L) / model.OPT(k)}")

# Example usage: Given beta, maximize alpha
alpha, beta = None, 0.45
model = LPModel(n, F, alpha, beta, outputflag = 0)
S, L, alpha, beta = model.solve()
if S is not None:
    for k in range(model.F.K):
        print(f" {k}: {model.ALG(k, L) / model.OPT(k)}")

Summary of distributions (*prior):
Support:
    [0.0, 0.00033238, 0.00056901, 0.00075408, 0.00090769, 0.0010409, 0.0011606, 0.0012713, 0.0013761, 0.0014776, 1.0, 100.0, 10000.0, 1000000.0, 100000000.0, 10000000000.0]
PMF:
 0: [0.71189, 0.070186, 0.04786, 0.037424, 0.030934, 0.026303, 0.022729, 0.019836, 0.017422, 0.015266, 9.9995e-05, 3.3332e-05, 1.1111e-05, 3.7035e-06, 1.2345e-06, 4.115e-07]
 1: [0.71189, 0.070186, 0.04786, 0.037424, 0.030934, 0.026303, 0.022729, 0.019836, 0.017422, 0.015266, 9.9995e-05, 3.3332e-05, 1.1111e-05, 3.7035e-06, 1.2345e-06, 0.0]
 2: [0.71189, 0.070186, 0.04786, 0.037424, 0.030934, 0.026303, 0.022729, 0.019836, 0.017422, 0.015266, 9.9995e-05, 3.3332e-05, 1.1111e-05, 3.7035e-06, 0.0, 0.0]
 3: [0.71189, 0.070186, 0.047861, 0.037424, 0.030934, 0.026303, 0.022729, 0.019836, 0.017422, 0.015267, 9.9996e-05, 3.3332e-05, 1.1111e-05, 0.0, 0.0, 0.0]
 4: [0.7119, 0.070187, 0.047861, 0.037424, 0.030934, 0.026303, 0.022729, 0.019836, 0.017422, 0.015267, 9.9997e-05, 3.333